# *ChatBot*
### *Álvaro Salís*

## 1. Importación de librerías

* Vamos a utilizar el cliente de openai sobre el que nos basaremos para completar el objetico de este trabajo.
* Panel nos ayudará a crear una interfaz visible y en este caso estilable.
* Con ayuda de json validaremos el json resultante del pedido





In [28]:

!pip install jupyter_bokeh
import openai
import panel as pn
import json
from openai import OpenAI

client = OpenAI(api_key="YOUR-API")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


He querido estilar el panel y con la versión de panel que he instalado no me ha dejado utilizar 'styles' pero si he podido meter unos estilos que pedí en 'duck.ai' en el atributo raw_css sobre el config del 'pn'.

In [29]:
custom_styles = """
:root {
    --primary-color: #FF6B35;
    --secondary-color: #004E89;
    --background-light: #F7F9FC;
    --text-dark: #2E2E2E;
    --shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}

.bk-input {
    font-size: 14px;
    padding: 12px 15px;
    border-radius: 8px;
    border: 2px solid #E0E0E0;
    transition: all 0.3s ease;
}

.bk-input:focus {
    border-color: var(--primary-color);
    box-shadow: var(--shadow);
    outline: none;
}

.bk-btn {
    font-weight: 600;
    letter-spacing: 0.5px;
    border-radius: 8px;
    padding: 10px 20px;
    transition: all 0.3s ease;
    text-transform: uppercase;
    font-size: 13px;
}

.bk-btn-primary {
    background-color: var(--primary-color) !important;
    border-color: var(--primary-color) !important;
}

.bk-btn-primary:hover {
    background-color: #E05A2B !important;
    transform: translateY(-2px);
}

.bk-btn:not(.bk-btn-primary) {
    background-color: white !important;
    color: var(--secondary-color) !important;
    border: 2px solid var(--secondary-color) !important;
}

.bk-btn:not(.bk-btn-primary):hover {
    background-color: var(--background-light) !important;
}
.dark-container {
    background-color: #222222;
    color: #ffffff;
    padding: 20px;
    border-radius: 10px;
}

.dark-title {
    color: #ffffff !important;
}

.bk-input {
    background-color: #333333 !important;
    color: #ffffff !important;
    border-color: #444444 !important;
}
.dark-container {
    background-color: #222222;
    color: #ffffff;
    padding: 20px;
    border-radius: 10px;
}

.dark-title {
    color: #ffffff !important;
}

.bk-input {
    background-color: #333333 !important;
    color: #ffffff !important;
    border-color: #444444 !important;
}
.user-prompt {
    background-color: #14d1ff;
    color: #ffffff !important;
    padding: 5px;
    border-radius: 5px;
    margin-bottom: 20px;
    max-width: 100%;
    min-width: 100%;
}
.assistant-prompt {
    background-color: #d9f1ff;
    color: #ffffff !important;
    padding: 5px;
    border-radius: 5px;
    margin-bottom: 20px;
    max-width: 100%;
    min-width: 100%;
}
.panel{
  margin-left:15px;
  margin-top:15px;
}

"""

pn.config.raw_css = [custom_styles]

## 2. Creación del contexto

Este sería un ejemplo de carta que vamos a utilizar como contexto inicial para este chatbot. Más adelante se le irán agregando a esta misma lista los distintos mensajes de la conversación.

In [30]:
mensajes = [
    {'role': 'system', 'content': '''
    Eres un asistente de chat para un restaurante llamado "Grosso Salís".
    Tu trabajo es ayudar a los clientes a hacer pedidos de comida para entrega a domicilio o recoger en el local.

    El menú incluye:
    - Pizzas: Margarita (€10), Pepperoni (€12), Hawaiana (€11), Vegetal (€10)
      Tamaños: Pequeña (-€2), Mediana (precio base), Grande (+€2)
    - Ingredientes extra: Queso (€1.5), Champiñones (€1), Jamón (€2), Aceitunas (€1)
    - Bebidas: Refresco (€2), Agua (€1.5), Cerveza (€3)

    Pregunta al cliente si quiere entrega a domicilio o recoger en el local.
    Si es entrega, pide la dirección.
    Finalmente, resume el pedido en formato JSON con los campos especificados.
    '''}
]

panels = []

Esta función simplemente se ocupa de mandar los requests al modelo de openai 'gpt-3.5-turbo' y de devolvernos las respuestas que se generen.

In [31]:
def obtener_completion(mensajes, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=mensajes,
        temperature=0.8,
    )
    return response.choices[0].message.content

Cada vez que mandemos un mensaje, esta función cargará la respuesta del modelo y la hará visible en el panel al que más adelante vincularé con el método 'bind'.

In [32]:
def collect_messages(event):
    global mensajes
    prompt = inp.value_input
    inp.value = ''

    mensajes.append({'role': 'user', 'content': prompt})
    respuesta = obtener_completion(mensajes)
    mensajes.append({'role': 'assistant', 'content': respuesta})

    panels.append(pn.Row('Usuario:', pn.pane.Markdown(prompt, width=600), css_classes=['user-prompt']))
    panels.append(pn.Row('Asistente:', pn.pane.Markdown(respuesta, width=600),css_classes=['assistant-prompt']))

    return pn.Column(*panels)

Está función es la más importante de todas. Al presionar el botón de *'Finalizar conversación'*, vamos a buscar entre todos los mensajes aquellos que sean del modelo y contengan una apertura de llave, normalmente solo habrá uno que será el último (Prodría haber cogido directamente *'mensajes[-1]'* pero así nos aseguramos mejor).

Si logramos capturar un json válido, gracias a un pequeño fragmento de html se puede programar un script para descargar el .json para su posterior procesamiento de comanda.

In [33]:
def end_chat(event):
    global mensajes

    json_str = None
    for msg in reversed(mensajes):
        if msg['role'] == 'assistant' and '{' in msg['content']:
            try:
                content = msg['content']
                json_str = content[content.find('{'):content.rfind('}')+1]
                json.loads(json_str)
                break
            except:
                continue

    if json_str:
        from IPython.display import HTML
        import base64

        b64 = base64.b64encode(json_str.encode()).decode()

        html = f'''
        <a id="download" href="data:application/json;base64,{b64}"
           download="resumen_pedido.json" style="display:none;"></a>
        <script>
            document.getElementById('download').click();
        </script>
        '''
        display(HTML(html))

        return json_str
    return None

Aquí colocamos todas las partes agregando también las vinculaciones y escuchas de eventos correspondientes.

In [34]:
pn.extension()

inp = pn.widgets.TextInput(
    value="Me gustaría una pizza barbacoa",
    placeholder='Mensaje',
    css_classes=['custom-input']
)

button_conversation = pn.widgets.Button(
    name="Enviar",
    css_classes=['hover-effect']
)

button_end_chat = pn.widgets.Button(
    name="Finalizar pedido",
    css_classes=['primary-button']
)

header = pn.pane.Markdown("""
    # 🍕 Grosso Salís - Pedidos a Domicilio
    Bienvenido a nuestro sistema de pedidos. Por favor, indícanos qué te gustaría pedir.
    """, css_classes=['dark-title'])

interactive_conversation = pn.bind(collect_messages, button_conversation)
button_end_chat.on_click(end_chat)


dashboard = pn.Column(
    header,
    inp,
    pn.Row(button_conversation, button_end_chat),
    pn.panel(interactive_conversation, loading_indicator=True, height=5000, css_classes=['panel']),
    sizing_mode="stretch_width",
    css_classes=['dark-container']
)

No mostramos un warning molesto de jupyter bokeh

In [35]:
import warnings
from bokeh.util.warnings import BokehUserWarning

warnings.filterwarnings("ignore", category=BokehUserWarning, message=".*reference already known.*")

## 3. Caso de uso

A partir de aquí ya se puede probar (He puesto mucho alto al panel para que no desborden los mensajes, así que hay que subirlo para arriba)

In [36]:
dashboard

Column(css_classes=['dark-container'], sizing_mode='stretch_width')
    [0] Markdown(str, css_classes=['dark-title'])
    [1] TextInput(css_classes=['custom-input'], placeholder='Mensaje')
    [2] Row
        [0] Button(css_classes=['hover-effect'], name='Enviar')
        [1] Button(css_classes=['primary-button'], name='Finalizar pedido')
    [3] ParamFunction(function, _pane=Column, css_classes=['panel'], defer_load=False, height=5000, loading_indicator=True)